<a href="https://colab.research.google.com/github/ShreyaJadhav9700/Amazon-sales-report-Dashboard/blob/main/Assignment_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load training data
customers = pd.read_csv('/content/train_customers.csv')
locations = pd.read_csv('/content/train_locations.csv')
orders = pd.read_csv('/content/train_orders.csv')

# Load test data
test_customers = pd.read_csv('/content/test_customers.csv')
test_locations = pd.read_csv('/content/test_locations.csv')

# Load vendor info
vendors = pd.read_csv('/content/vendors.csv')

/tmp/ipython-input-3255880015.py:6: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('/content/train_orders.csv')


In [2]:
import pandas as pd
import numpy as np

# Make sure dob is parsed as datetime
customers['dob'] = pd.to_datetime(customers['dob'], errors='coerce')

# Set reference year manually as 2025
customers['age'] = 2025 - customers['dob'].dt.year

In [3]:
# View dimensions and nulls
for name, df in [("customers", customers),
                 ("locations", locations),
                 ("orders", orders),
                 ("vendors", vendors),
                 ("test_customers", test_customers),
                 ("test_locations", test_locations)]:
    print(f"\n{name.upper()} — Shape: {df.shape}")
    print(df.info())
    print(df.head())


CUSTOMERS — Shape: (34674, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34674 entries, 0 to 34673
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  34674 non-null  object        
 1   gender       22520 non-null  object        
 2   dob          3046 non-null   datetime64[ns]
 3   status       34674 non-null  int64         
 4   verified     34674 non-null  int64         
 5   language     21099 non-null  object        
 6   created_at   34674 non-null  object        
 7   updated_at   34674 non-null  object        
 8   age          3046 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 2.4+ MB
None
  customer_id gender dob  status  verified language       created_at  \
0     TCHWPBT   Male NaT       1         1       EN   2/7/2023 19:16   
1     ZGFSYCZ   Male NaT       1         1       EN   2/9/2023 12:04   
2     S2ALZFL   Male

In [4]:
# Merge orders with location info
orders_with_location = pd.merge(orders, locations, on='customer_id', how='left')
orders_with_location.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR,location_number,location_type,latitude,longitude
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84,0,Work,-0.090650,-78.580196
1,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84,1,Home,-0.676098,-78.511007
2,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84,2,NaN,-96.407541,43.557974
3,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84,3,NaN,-0.089966,0.874226
4,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,NaN,No,...,8/1/2024 5:30,78,8/2/2024 5:34,0,Home,H5LGGFX X 0 X 78,0,Home,1.733950,-78.795830


In [5]:
# Merge Orders with customers
orders_cust = pd.merge(orders_with_location, customers, on='customer_id', how='left')

In [6]:

# orders_cust with vendors
orders_full = pd.merge(orders_cust, vendors, left_on='vendor_id', right_on='id', how='left')
orders_full = orders_full.drop(columns=['id'])

In [7]:
print("Orders Full Columns:\n", orders_full.columns)
print("\nSample Data:\n", orders_full.head())

Orders Full Columns:
 Index(['order_id', 'customer_id', 'item_count', 'grand_total', 'payment_mode',
       'promo_code', 'vendor_discount_amount',
       'promo_code_discount_percentage', 'is_favorite', 'is_rated',
       'vendor_rating_x', 'driver_rating', 'deliverydistance',
       'preparationtime', 'delivery_time', 'order_accepted_time',
       'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time',
       'delivered_time', 'delivery_date', 'vendor_id', 'created_at_x',
       'LOCATION_NUMBER', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR',
       'location_number', 'location_type', 'latitude_x', 'longitude_x',
       'gender', 'dob', 'status_x', 'verified_x', 'language_x', 'created_at_y',
       'updated_at_x', 'age', 'authentication_id', 'latitude_y', 'longitude_y',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_haked_delivering',
       

In [8]:
# STEP 1: Define selected columns based on the test dataset
selected_cols = [
    'customer_id',
    'gender',
    'age',  # derived from dob
    'location_number',
    'location_type',
    'latitude',
    'longitude',
    'vendor_id'
]


# --- Already loaded and merged data into orders_full ---

# Rename latitude and longitude from locations data because we want locations of customers
orders_full.rename(columns={
    'latitude_x': 'latitude',
    'longitude_x': 'longitude',
    'language_x': 'language'
}, inplace=True)

# Filter only the required columns that match test schema
train_data = orders_full[selected_cols]

# Optional: check for nulls or inconsistencies
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395867 entries, 0 to 395866
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   customer_id      395867 non-null  object 
 1   gender           316100 non-null  object 
 2   age              35822 non-null   float64
 3   location_number  395867 non-null  int64  
 4   location_type    209914 non-null  object 
 5   latitude         395801 non-null  float64
 6   longitude        395801 non-null  float64
 7   vendor_id        395867 non-null  int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 24.2+ MB
None


In [9]:
print(train_data.head())

  customer_id gender  age  location_number location_type   latitude  \
0     KL09J9N   Male  NaN                0          Work  -0.090650   
1     KL09J9N   Male  NaN                1          Home  -0.676098   
2     KL09J9N   Male  NaN                2           NaN -96.407541   
3     KL09J9N   Male  NaN                3           NaN  -0.089966   
4     H5LGGFX   Male  NaN                0          Home   1.733950   

   longitude  vendor_id  
0 -78.580196         84  
1 -78.511007         84  
2  43.557974         84  
3   0.874226         84  
4 -78.795830         78  


In [10]:

unique_vendors = train_data['vendor_id'].unique()
unique_vendors

array([ 84,  78,   4, 157, 160,  43, 294, 105, 104, 243,  83, 159,  55,
       231,  81, 300, 180,  13,  20, 303, 310, 289, 201, 265,  66, 192,
       145, 113, 134, 106, 110,  75,  82, 149, 176,  85, 154, 203, 274,
       115, 225, 193,  33,  92, 196, 221, 299, 207,  23, 288,  76, 161,
       298,  90, 199, 195, 271, 148, 304, 188, 259,  28, 197,  44, 189,
        79, 250, 191, 237, 356, 419, 391,  67,  86, 537, 459, 386, 295,
       401, 398, 216, 573, 575, 577, 578, 582, 583, 547, 679, 676, 681,
       841, 846, 858, 849, 845, 843, 856, 855, 907])

In [11]:
cust_locs = train_data[['customer_id', 'location_number', 'gender', 'age', 'location_type', 'latitude', 'longitude']].drop_duplicates()
cust_locs.head()

,customer_id,location_number,gender,age,location_type,latitude,longitude
0,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196
1,KL09J9N,1,Male,NaN,Home,-0.676098,-78.511007
2,KL09J9N,2,Male,NaN,NaN,-96.407541,43.557974
3,KL09J9N,3,Male,NaN,NaN,-0.089966,0.874226
4,H5LGGFX,0,Male,NaN,Home,1.733950,-78.795830


In [12]:

import itertools

# Create a cartesian product: every (cust_loc) × every vendor
cust_loc_vendor = cust_locs.assign(key=1).merge(
    pd.DataFrame(unique_vendors, columns=['vendor_id']).assign(key=1),
    on='key'
).drop('key', axis=1)

cust_loc_vendor

,customer_id,location_number,gender,age,location_type,latitude,longitude,vendor_id
0,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196,84
1,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196,78
2,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196,4
3,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196,157
4,KL09J9N,0,Male,NaN,Work,-0.090650,-78.580196,160
...,...,...,...,...,...,...,...,...
5042995,E7QP3CG,0,Male,NaN,NaN,-0.127822,-78.575803,845
5042996,E7QP3CG,0,Male,NaN,NaN,-0.127822,-78.575803,843
5042997,E7QP3CG,0,Male,NaN,NaN,-0.127822,-78.575803,856
5042998,E7QP3CG,0,Male,NaN,NaN,-0.127822,-78.575803,855


In [13]:

# Actual orders in your training data
actual_orders = train_data[['customer_id', 'location_number', 'vendor_id']]
actual_orders['target'] = 1
actual_orders.head()

/tmp/ipython-input-841651819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_orders['target'] = 1


,customer_id,location_number,vendor_id,target
0,KL09J9N,0,84,1
1,KL09J9N,1,84,1
2,KL09J9N,2,84,1
3,KL09J9N,3,84,1
4,H5LGGFX,0,78,1


In [14]:
# Merge to get target
final_train = cust_loc_vendor.merge(
    actual_orders,
    on=['customer_id', 'location_number', 'vendor_id'],
    how='left'
)

# Fill NaNs in target (meaning no order was placed) with 0
final_train['target'] = final_train['target'].fillna(0).astype(int)

In [15]:
final_train.head()


,customer_id,location_number,gender,age,location_type,latitude,longitude,vendor_id,target
0,KL09J9N,0,Male,NaN,Work,-0.09065,-78.580196,84,1
1,KL09J9N,0,Male,NaN,Work,-0.09065,-78.580196,78,0
2,KL09J9N,0,Male,NaN,Work,-0.09065,-78.580196,4,0
3,KL09J9N,0,Male,NaN,Work,-0.09065,-78.580196,157,0
4,KL09J9N,0,Male,NaN,Work,-0.09065,-78.580196,160,0


In [16]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5252442 entries, 0 to 5252441
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   customer_id      object 
 1   location_number  int64  
 2   gender           object 
 3   age              float64
 4   location_type    object 
 5   latitude         float64
 6   longitude        float64
 7   vendor_id        int64  
 8   target           int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 360.7+ MB


In [17]:

# Negative Downsampling
# Step 1: Separate positives and negatives
positives = final_train[final_train['target'] == 1]
negatives = final_train[final_train['target'] == 0]

# Step 2: Downsample negatives to match number of positives (or a fixed ratio)
neg_sample = negatives.sample(n=len(positives)*2, random_state=42)

# Step 3: Combine
balanced_train = pd.concat([positives, neg_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

In [18]:

# Step 4: Verify class balance
print(balanced_train['target'].value_counts())

target
0    791960
1    395980
Name: count, dtype: int64


In [19]:

balanced_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187940 entries, 0 to 1187939
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   customer_id      1187940 non-null  object 
 1   location_number  1187940 non-null  int64  
 2   gender           886509 non-null   object 
 3   age              105491 non-null   float64
 4   location_type    648630 non-null   object 
 5   latitude         1187772 non-null  float64
 6   longitude        1187772 non-null  float64
 7   vendor_id        1187940 non-null  int64  
 8   target           1187940 non-null  int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 81.6+ MB


In [20]:
balanced_train = balanced_train.drop(columns=['age']) # Very high missingness — likely not useful

balanced_train['gender'] = balanced_train['gender'].fillna('unknown')
balanced_train['location_type'] = balanced_train['location_type'].fillna('unknown')

In [21]:

balanced_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187940 entries, 0 to 1187939
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   customer_id      1187940 non-null  object 
 1   location_number  1187940 non-null  int64  
 2   gender           1187940 non-null  object 
 3   location_type    1187940 non-null  object 
 4   latitude         1187772 non-null  float64
 5   longitude        1187772 non-null  float64
 6   vendor_id        1187940 non-null  int64  
 7   target           1187940 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 72.5+ MB


In [22]:

df = balanced_train

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Drop customer_id (not useful for prediction)
# df = df.drop(columns=['customer_id']) # This line is removed

# Step 2: Label encode categorical features
label_encoders = {}
for col in ['gender', 'location_type']:
    le = LabelEncoder()

    # Combine training and test data for fitting the encoder
    combined_col_data = pd.concat([df[col], test_cross[col]]).astype(str).fillna('unknown')

    # Fit the encoder on all unique values across combined data after filling nulls
    le.fit(combined_col_data.unique())

    # Transform both training and test data
    df[col] = le.transform(df[col].astype(str).fillna('unknown'))
    label_encoders[col] = le  # Save encoders for later use

# Step 3: Split into features and target
X = df.drop(columns=['target'])
y = df['target']

# Optional: Handle any residual missing values (in latitude/longitude)
X = X.fillna(X.mean())

# Step 4: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 5: Train XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Step 6: Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [15:10:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.7840000336717342
              precision    recall  f1-score   support

           0       0.80      0.90      0.85    158392
           1       0.73      0.56      0.63     79196

    accuracy                           0.78    237588
   macro avg       0.77      0.73      0.74    237588
weighted avg       0.78      0.78      0.78    237588



In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Step 1: Merge test data on customer_id
test_merged = test_customers.merge(test_locations, on='customer_id', how='left')

# Step 2: Create vendor_id DataFrame from training data
vendor_ids = df['vendor_id'].unique()  # 'df' is your training DataFrame
vendor_df = pd.DataFrame({'vendor_id': vendor_ids})

# Step 3: Add key columns for cross join
test_merged['key'] = 1
vendor_df['key'] = 1

# Step 4: Perform cross join
test_cross = test_merged.merge(vendor_df, on='key').drop('key', axis=1)

# Step 5: Fill missing latitude/longitude with column mean
test_cross['latitude'] = test_cross['latitude'].fillna(test_cross['latitude'].mean())
test_cross['longitude'] = test_cross['longitude'].fillna(test_cross['longitude'].mean())

# Step 6: Encode 'gender' and 'location_type' safely
for col in ['gender', 'location_type']:
    le = label_encoders[col]  # Loaded from training step
    test_cross[col] = test_cross[col].fillna('unknown')

    # Replace unseen labels with 'unknown'
    test_cross[col] = test_cross[col].apply(lambda x: x if x in le.classes_ else 'unknown')

    # If 'unknown' wasn't in the original training classes, add it now
    if 'unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'unknown')

    # Transform using updated LabelEncoder
    test_cross[col] = le.transform(test_cross[col])


In [32]:
test_cross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633200 entries, 0 to 1633199
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   customer_id      1633200 non-null  object 
 1   gender           1153500 non-null  object 
 2   dob              142700 non-null   float64
 3   status           1633200 non-null  int64  
 4   verified         1633200 non-null  int64  
 5   language         1194700 non-null  object 
 6   created_at       1633200 non-null  object 
 7   updated_at       1633200 non-null  object 
 8   location_number  1633100 non-null  float64
 9   location_type    886800 non-null   object 
 10  latitude         1633200 non-null  float64
 11  longitude        1633200 non-null  float64
 12  vendor_id        1633200 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 162.0+ MB


In [33]:
drop_cols = ['customer_id', 'dob', 'status', 'verified', 'language', 'created_at', 'updated_at']


In [34]:
# Now test_cross is ready for model prediction
X_test = test_cross.drop(columns=drop_cols)  # keep only features
X_test = X_test[X_train.columns]
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633200 entries, 0 to 1633199
Data columns (total 6 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   location_number  1633100 non-null  float64
 1   gender           1153500 non-null  object 
 2   location_type    886800 non-null   object 
 3   latitude         1633200 non-null  float64
 4   longitude        1633200 non-null  float64
 5   vendor_id        1633200 non-null  int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 74.8+ MB


In [41]:
# Step 1: Encode categorical columns
for col in ['gender', 'location_type']:
    le = label_encoders[col]
    test_cross[col] = test_cross[col].fillna('unknown')
    test_cross[col] = test_cross[col].apply(lambda x: x if x in le.classes_ else 'unknown')

    if 'unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'unknown')

    test_cross[col] = le.transform(test_cross[col])

# Step 2: Prepare test features in correct column order
X_test = test_cross[model.feature_names_in_]  # Ensures exact match in name + order

# Step 3: Predict
y_pred = model.predict(X_test)

# Step 4: Format output
test_cross['target'] = y_pred
test_cross['CID X LOC_NUM X VENDOR'] = (
    test_cross['customer_id'] + ' X ' +
    test_cross['location_number'].astype(str) + ' X ' +
    test_cross['vendor_id'].astype(str)
)

# Step 5: Final submission format
submission = test_cross[['CID X LOC_NUM X VENDOR', 'target']]


In [42]:

submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633200 entries, 0 to 1633199
Data columns (total 2 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   CID X LOC_NUM X VENDOR  1633200 non-null  object
 1   target                  1633200 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.9+ MB


In [43]:
# Get only positive predictions
recommendations = submission[submission['target'] == 1]

# View sample recommendations
print(recommendations.head())

   CID X LOC_NUM X VENDOR  target
14    ICE2DJP X 0.0 X 681       1
24    ICE2DJP X 0.0 X 193       1
29    ICE2DJP X 0.0 X 386       1
54    ICE2DJP X 0.0 X 843       1
64    ICE2DJP X 0.0 X 846       1


In [44]:

recommendations.to_csv('recommendations.csv', index=False)

In [45]:
submission.to_csv('submission.csv', index=False)